In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocess the Amazon product reviews data.

Download the Dataset from kaggle

In [ ]:
!kaggle datasets download -d bittlingmayer/amazonreviews


Dataset URL: https://www.kaggle.com/datasets/bittlingmayer/amazonreviews
License(s): unknown
 98% 482M/493M [00:04<00:00, 105MB/s]
100% 493M/493M [00:04<00:00, 118MB/s]


In [ ]:
# Unzip the dataset
!unzip /content/drive/MyDrive/amazonreviews-dataset/amazonreviews.zip -d /content/drive/MyDrive/amazonreviews-dataset

Archive:  /content/drive/MyDrive/amazonreviews-dataset/amazonreviews.zip
  inflating: /content/drive/MyDrive/amazonreviews-dataset/test.ft.txt.bz2  
  inflating: /content/drive/MyDrive/amazonreviews-dataset/train.ft.txt.bz2  


# Implement and fine-tune an LLM using NVIDIA NeMo for sentiment classification.

Kami tidak menggunakan NVIDIA’s NLP models.
Model nvidia/Aegis-AI-Content-Safety-LlamaGuard-Defensive-1.0 difokuskan untuk content safety atau moderasi konten, bukan untuk sentiment analysis atau analisis review produk. Model ini dirancang oleh NVIDIA untuk mendeteksi dan memoderasi konten yang tidak aman atau sensitif, seperti bahasa yang tidak pantas, konten berbahaya, atau bahasa yang melanggar aturan komunitas

Untuk analisis review produk, terutama untuk sentimen atau pemahaman konteks pengguna, model yang lebih cocok adalah model yang dilatih untuk sentiment analysis atau analisis teks yang fokus pada ulasan pelanggan. Beberapa pilihan alternatif yang bisa lebih sesuai adalah:

Sentiment Analysis Model dari Hugging Face:


* nlptown/bert-base-multilingual-uncased-sentiment: BERT model yang mendukung beberapa bahasa dan cocok untuk analisis sentimen, baik positif maupun negatif.
* distilbert-base-uncased-finetuned-sst-2-english: Versi lebih ringan dari BERT yang dirancang untuk mengklasifikasikan teks dalam dua kelas, positif dan negatif, dan sering digunakan untuk review produk.

In [ ]:
!pip install transformers pandas


Load and Decompress the Dataset

In [ ]:
import pandas as pd
import bz2
from transformers import pipeline
from sklearn.metrics import classification_report

gdrive_dir = '/content/drive/MyDrive/amazonreviews-dataset/'
train_dir = gdrive_dir + 'train.ft.txt.bz2'
test_dir = gdrive_dir + 'test.ft.txt.bz2'

with bz2.open(train_dir, 'rt') as f:
    data = f.readlines()

# with bz2.BZ2File(train_dir, 'rb') as f:
#     data = f.read().decode('utf-8').splitlines()

# Check the total lines and the first few entries
print("Total lines in dataset:", len(data))
for line in data[:5]:
    print(line.strip())

# Prepare valid reviews
reviews = []
for line in data:
    # Split the line on the first space to separate the label from the review
    parts = line.strip().split(' ', 1)  # Split only on the first space
    if len(parts) == 2:  # Check if the line has the expected number of columns
        reviews.append(parts)

# Create DataFrame from valid reviews
df = pd.DataFrame(reviews, columns=['sentiment', 'reviewText'])

# Check the resulting DataFrame
print("DataFrame shape:", df.shape)
print("DataFrame head:")
print(df.head())

Preprocess the Data Clean and label the sentiment based on the dataset. The first column usually indicates the sentiment label.

In [4]:
# Check the unique sentiment labels
#print(df['sentiment'].unique())

# Map sentiments to a simpler format
sentiment_mapping = {
    '__label__1': 'negative',
    '__label__2': 'positive'
}

df['mapped_sentiment'] = df['sentiment'].map(sentiment_mapping)
print(df['mapped_sentiment'])


0          positive
1          positive
2          positive
3          positive
4          positive
             ...   
3599995    negative
3599996    negative
3599997    negative
3599998    negative
3599999    positive
Name: mapped_sentiment, Length: 3600000, dtype: object


Load the Pre-trained Sentiment Analysis Model Use the nlptown/bert-base-multilingual-uncased-sentiment model for sentiment analysis.

In [5]:
from transformers import pipeline

# Load the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Utilize prompt engineering to optimize sentiment analysis accuracy.

Sebelum melakukan analisis sentimen, sangat penting untuk membersihkan teks ulasan. Ini termasuk menghapus karakter khusus, menghapus kata-kata yang tidak penting (stopwords), dan melakukan stemming atau lemmatization. Dengan pembersihan yang baik, model dapat lebih akurat dalam menentukan sentimen.

In [ ]:
import re
from nltk.corpus import stopwords

def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Menghapus karakter non-alfabet
    text = text.lower()  # Mengubah ke huruf kecil
    text = ' '.join(word for word in text.split() if word not in stopwords.words('english'))  # Menghapus stopwords
    return text

df['cleaned_review'] = df['reviewText'].apply(clean_text)


Analyze Sentiment Use the model to classify the sentiment of the reviews.


In [ ]:
# Prepare the reviews for sentiment analysis
review_texts = df['cleaned_review'].tolist()
print(review_texts)

# Analyze sentiment
predictions = sentiment_pipeline(review_texts)

# Extract sentiment labels and scores
df['predicted_sentiment'] = [pred['label'] for pred in predictions]
df['predicted_score'] = [pred['score'] for pred in predictions]

# Define your labels explicitly if necessary
target_names = ['negative', 'positive']  # Modify based on your classes



# Evaluate the model’s performance using metrics like accuracy, precision, recall, and F1-score.

Menggunakan fungsi classification_report dari sklearn.metrics. Ini adalah fungsi yang memungkinkan Anda untuk menghitung dan menampilkan metrik evaluasi untuk model klasifikasi.

Penjelasan:
classification_report: Fungsi ini mengambil dua argumen:

reviews['sentiment']: Ini adalah label sentimen aktual yang telah ditentukan sebelumnya berdasarkan penilaian keseluruhan.
reviews['mapped_sentiment']: Ini adalah label sentimen yang diprediksi oleh model berdasarkan ulasan yang dianalisis.
Metrik yang Dihasilkan: Fungsi ini menghasilkan laporan yang mencakup:

Akurasi: Persentase prediksi yang benar dari total prediksi.
Presisi: Rasio prediksi positif yang benar terhadap total prediksi positif.
Perolehan (Recall): Rasio prediksi positif yang benar terhadap total aktual positif.
Skor F1: Rata-rata harmonis dari presisi dan perolehan, memberikan gambaran umum tentang keseimbangan antara keduanya.


In [ ]:
# Evaluate the model
print("Mapped Sentiment:", df['mapped_sentiment'].unique())
print("Predicted Sentiment:", df['predicted_sentiment'].unique())
#classification_report: This function provides precision, recall, F1-score, and support for each class.
print(classification_report(df['mapped_sentiment'], df['predicted_sentiment'], target_names=target_names))

# Display the first few results
print(df[['reviewText', 'mapped_sentiment', 'predicted_sentiment', 'predicted_score']].head())

# Calculate and print overall accuracy
#accuracy_score: This function calculates the overall accuracy of the model.
accuracy = accuracy_score(df['mapped_sentiment'], df['predicted_sentiment'])
print("Accuracy:", accuracy)

In [ ]:
# Save the results to a CSV file
df.to_csv('amazon_reviews_with_sentiment.csv', index=False)


# Visualize sentiment distribution and key insights.

Distribusi Sentimen Aktual dan Prediksi
Visualisasi distribusi sentimen dapat memberikan gambaran tentang bagaimana data dan prediksi model tersebar. Diagram batang (bar plot) adalah salah satu cara efektif untuk melihat ini.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot distribusi sentimen aktual
plt.figure(figsize=(12, 6))
sns.countplot(x='sentiment', data=reviews, palette="viridis")
plt.title("Distribusi Sentimen Aktual")
plt.xlabel("Sentimen")
plt.ylabel("Jumlah")
plt.show()

# Plot distribusi sentimen yang diprediksi
plt.figure(figsize=(12, 6))
sns.countplot(x='mapped_sentiment', data=reviews, palette="viridis")
plt.title("Distribusi Sentimen yang Diprediksi")
plt.xlabel("Sentimen yang Diprediksi")
plt.ylabel("Jumlah")
plt.show()


# Assess and mitigate any biases in the sentiment analysis model



1. Penilaian Bias dalam Model


*   Analisis Distribusi Sentimen: Lakukan analisis pada data latih dan data prediksi untuk melihat apakah model cenderung memberikan prediksi tertentu lebih sering, misalnya lebih sering memberikan label "positif" dibanding "netral" atau "negatif".

*   Evaluasi Performa Berdasarkan Subkelompok Bahasa: nlptown/bert-base-multilingual-uncased-sentiment adalah model multibahasa, tetapi mungkin lebih efektif pada bahasa tertentu. Anda bisa menguji prediksi pada data berbahasa berbeda untuk melihat apakah terdapat bias dalam performa.



2. Menggunakan Metrik Keadilan dan Analisis Mendalam

Menggunakan Confusion Matrix untuk mengevaluasi keadilan. Ini akan membantu Anda melihat bagaimana model bekerja di seluruh kelas sentimen:



In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Evaluasi performa berdasarkan label prediksi
cm = confusion_matrix(reviews['sentiment'], reviews['predicted_sentiment'])
print(cm)
print(classification_report(reviews['sentiment'], reviews['predicted_sentiment']))


# Tools

Kami menggunakan General Transformers. Karena model yang kami pakai model berbasis transformers biasa/bukan class Nemo